The following code is from chunking evaluation from youtube video. Here as reference. I adopted this code for my purpose. Start running in lower cell where I wrote "#run this code starting here:..."

In [ ]:
from chunking_evaluation.chunking import FixedTokenChunker, RecursiveTokenChunker, ClusterSemanticChunker, LLMSemanticChunker, KamradtModifiedChunker
from chunking_evaluation import GeneralEvaluation, SyntheticEvaluation, BaseChunker
from chunking_evaluation.utils import openai_token_count
from chromadb.utils import embedding_functions
import pandas as pd
from IPython.display import display, clear_output
import http.client
import os

In [ ]:
# Load API key from either google colab secrets or manually enter it
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
# Uncomment the line below if you prefer to manually enter the key
# OPENAI_API_KEY = "YOUR_OPENAI_KEY"

In [ ]:
ef = embedding_functions.OpenAIEmbeddingFunction(api_key=OPENAI_API_KEY, model_name="text-embedding-3-large")

In [ ]:
def download_text(book_id, file_name, directory):
    conn = http.client.HTTPSConnection("www.gutenberg.org")
    url = f"/files/{book_id}/{book_id}-0.txt"

    conn.request("GET", url)
    response = conn.getresponse()

    if response.status == 200:
        text = response.read().decode('utf-8')

        # Create directory if it does not exist
        os.makedirs(directory, exist_ok=True)

        # Save the text to the specified file within the directory
        file_path = os.path.join(directory, file_name)
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(text)
        print(f"Book '{file_name}' downloaded and saved successfully in '{directory}'.")
    else:
        print(f"Failed to download the book. Status code: {response.status}")

# Define the books to download with their IDs and file names
books = {
    1661: "the_adventures_of_sherlock_holmes.txt",
    1342: "pride_and_prejudice.txt",
    174: "the_picture_of_dorian_gray.txt"
}

# Define the directory to save the books
directory = "corpora"

# Download each book
for book_id, file_name in books.items():
    download_text(book_id, file_name, directory)


In [ ]:
# Specify the corpora paths and output CSV file
corpora_paths = [
    'corpora/the_adventures_of_sherlock_holmes.txt',
    'corpora/pride_and_prejudice.txt',
    'corpora/the_picture_of_dorian_gray.txt',
]
queries_csv_path = 'generated_queries_and_excerpts.csv'

# Initialize the evaluation
evaluation = SyntheticEvaluation(corpora_paths, queries_csv_path, openai_api_key=OPENAI_API_KEY)


In [ ]:
# Generate queries and excerpts, and save to CSV
evaluation.generate_queries_and_excerpts(approximate_excerpts=True, num_rounds=1, queries_per_corpus=3)

In [ ]:
# Apply filter to remove queries with poor excerpts
evaluation.filter_poor_excerpts(threshold=0.36)

# Apply filter to remove duplicates
evaluation.filter_duplicates(threshold=0.6)

In [ ]:
# Define a custom chunking class
class CustomChunker(BaseChunker):
    def split_text(self, text):
        # Custom chunking logic
        return [text[i:i+1200] for i in range(0, len(text), 1200)]

In [ ]:
# Instantiate the custom chunker
chunker = CustomChunker()

# Run the evaluation on the filtered data
results = evaluation.run(chunker, embedding_function=ef)

# Remove detailed scores for brevity
del results['corpora_scores']

# Print results via pandas
df_results = pd.DataFrame([results])
df_results.head()

In [ ]:
#run this code starting here:
# ✅ Full Colab Notebook: True OpenAI Batch API for Ground Truth Generation from PDF (End-to-End Version)

# --- Install All Required Packages
!apt-get install -y poppler-utils
!pip install pymupdf
!pip install langchain langchain-community faiss-cpu transformers sentence-transformers
!pip install -U FlagEmbedding
!pip install openai chromadb
!pip install git+https://github.com/brandonstarxel/chunking_evaluation.git

# --- Import Libraries
import os
import pandas as pd
import time
import json
import openai
import torch
import faiss
from FlagEmbedding import BGEM3FlagModel
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from chunking_evaluation import SyntheticEvaluation






Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.7 [186 kB]
Fetched 186 kB in 1s (125 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126101 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.7_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.7) ...
Setting up poppler-utils (22.02.0-2ubuntu0.7) ...
Processing triggers for man-db (2.10.2-1) ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 113.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# --- Set Up API Key
from google.colab import userdata
OPENAI_API_KEY = userdata.get('openai_API_Key')
openai.api_key = OPENAI_API_KEY


In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:

# --- Load and Chunk Your PDF
pdf_path = '/content/drive/My Drive/ProstateCancer_2025.pdf'

loader = PyMuPDFLoader(pdf_path)
documents = loader.load()
print(f"Loaded {len(documents)} pages from PDF.")

splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=250)
all_chunks = []

for doc in documents:
    page_number = doc.metadata.get('page_number', None)
    splits = splitter.split_text(doc.page_content)
    for split in splits:
        all_chunks.append({
            'text': split,
            'page_number': page_number
        })

print(f"Total {len(all_chunks)} smart chunks created.")


Loaded 223 pages from PDF.
Total 230 smart chunks created.


In [ ]:
!pip install -U openai




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.2/661.2 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.75.0
    Uninstalling openai-1.75.0:
      Successfully uninstalled openai-1.75.0


In [ ]:
from openai import OpenAI
import json
import os
import time

client = OpenAI(api_key=OPENAI_API_KEY)

# --- Prepare batch requests ---
batch_requests = []

queries_per_chunk = 1
for idx, chunk in enumerate(all_chunks):
    prompt = (
        f"Given the following excerpt from a medical document:\n\n"
        f"\"\"\"\n{chunk['text']}\n\"\"\"\n\n"
        f"Please generate {queries_per_chunk} retrieval questions that could be answered using the excerpt, "
        f"and provide the corresponding exact answer excerpt for each question. "
        f"Respond in this JSON format:\n\n"
        f"[{{\"query\": \"...\", \"answer_excerpt\": \"...\"}}]"
    )

    batch_requests.append({
        "custom_id": f"chunk_{idx}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o",
            "messages": [
                {"role": "system", "content": "You are a helpful assistant that generates realistic information retrieval evaluation data."},
                {"role": "user", "content": prompt}
            ],
            "temperature": 0.5,
            "max_tokens": 800
        }
    })

# --- Split into mini-batches ---
def chunk_list(lst, chunk_size):
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

mini_batches = list(chunk_list(batch_requests, 40))  # 40 requests per batch

print(f"Total mini-batches created: {len(mini_batches)}")

# --- Submit each mini-batch separately ---
for batch_num, mini_batch in enumerate(mini_batches):
    batch_filename = f"/content/drive/My Drive/batch_requests_part_{batch_num}.jsonl"

    # Save mini-batch to JSONL
    with open(batch_filename, 'w') as f:
        for request in mini_batch:
            f.write(json.dumps(request) + "\n")

    print(f"Saved mini-batch {batch_num} with {len(mini_batch)} requests to {batch_filename}")

    # Upload mini-batch file
    with open(batch_filename, "rb") as f:
        upload_response = client.files.create(file=f, purpose="batch")
    file_id = upload_response.id
    print(f"Uploaded file ID for mini-batch {batch_num}: {file_id}")

    # Submit mini-batch job
    batch_response = client.batches.create(
        input_file_id=file_id,
        endpoint="/v1/chat/completions",
        completion_window="24h"
    )

    print(f"🚀 Mini-batch {batch_num} submitted:")
    print(f"Batch Job ID: {batch_response.id}")
    print(f"Batch Status: {batch_response.status}")
    print()

    # Optional: wait 2-3 seconds before next submission
    time.sleep(3)

print("✅ All mini-batches submitted successfully!")


Total mini-batches created: 6
Saved mini-batch 0 with 40 requests to /content/drive/My Drive/batch_requests_part_0.jsonl
Uploaded file ID for mini-batch 0: file-9GPz2E2YFLYU9cboRN7p8j
🚀 Mini-batch 0 submitted:
Batch Job ID: batch_680eb9bbafc881909924ed00e4829b61
Batch Status: validating

Saved mini-batch 1 with 40 requests to /content/drive/My Drive/batch_requests_part_1.jsonl
Uploaded file ID for mini-batch 1: file-JfdDLJ5NCCMRYtqJQPJdt9
🚀 Mini-batch 1 submitted:
Batch Job ID: batch_680eb9bf94788190b25b968b105379f6
Batch Status: validating

Saved mini-batch 2 with 40 requests to /content/drive/My Drive/batch_requests_part_2.jsonl
Uploaded file ID for mini-batch 2: file-UPJU9N8iybi9Ny4dkeaDNL
🚀 Mini-batch 2 submitted:
Batch Job ID: batch_680eb9c4b0108190ad7691e7aeaaecc8
Batch Status: validating

Saved mini-batch 3 with 40 requests to /content/drive/My Drive/batch_requests_part_3.jsonl
Uploaded file ID for mini-batch 3: file-Cq2tNPFfSTfScjcp5aLG9A
🚀 Mini-batch 3 submitted:
Batch Job ID:

In [ ]:
from openai import OpenAI
import time
import requests
import os

client = OpenAI(api_key=OPENAI_API_KEY)

# List your mini batch IDs
mini_batch_ids = [
    "batch_680eb9bbafc881909924ed00e4829b61",
    "batch_680eb9bf94788190b25b968b105379f6",
    "batch_680eb9c4b0108190ad7691e7aeaaecc8",
    "batch_680eb9c8c70481909ff883c0a37025fb",
    "batch_680eb9cc613881908ea29327189641d5",
    "batch_680eb9d03acc8190af68889d01203b0a"
]

# Make sure the output directory exists
output_dir = "/content/drive/My Drive/batch_outputs/"
os.makedirs(output_dir, exist_ok=True)

# Poll and download outputs
for batch_id in mini_batch_ids:
    print(f"⏳ Waiting for batch {batch_id} to complete...")

    while True:
        batch_status = client.batches.retrieve(batch_id)
        status = batch_status.status
        print(f"Batch {batch_id} - Current status: {status}")

        if status in ["completed", "failed", "cancelled"]:
            break

        time.sleep(30)  # wait 30 seconds between checks

    if status == "completed":
        print(f"✅ Batch {batch_id} completed!")
        if batch_status.output_file_id:
            output_file_id = batch_status.output_file_id
            # Build the download URL
            url = f"https://api.openai.com/v1/files/{output_file_id}/content"
            headers = {
                "Authorization": f"Bearer {OPENAI_API_KEY}"
            }
            # Download the file
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                # Save it into Google Drive
                save_path = os.path.join(output_dir, f"batch_output_{output_file_id}.jsonl")
                with open(save_path, "wb") as f:
                    f.write(response.content)
                print(f"✅ File downloaded and saved as {save_path}")
            else:
                print(f"❌ Failed to download file for batch {batch_id}. Status: {response.status_code}")
                print(response.text)
        else:
            print(f"⚠️ No output file found for batch {batch_id}.")
    else:
        print(f"❌ Batch {batch_id} failed or was cancelled.")


⏳ Waiting for batch batch_680eb9bbafc881909924ed00e4829b61 to complete...
Batch batch_680eb9bbafc881909924ed00e4829b61 - Current status: completed
✅ Batch batch_680eb9bbafc881909924ed00e4829b61 completed!
✅ File downloaded and saved as /content/drive/My Drive/batch_outputs/batch_output_file-FH1eoSrABx5psioAxKPegR.jsonl
⏳ Waiting for batch batch_680eb9bf94788190b25b968b105379f6 to complete...
Batch batch_680eb9bf94788190b25b968b105379f6 - Current status: completed
✅ Batch batch_680eb9bf94788190b25b968b105379f6 completed!
✅ File downloaded and saved as /content/drive/My Drive/batch_outputs/batch_output_file-YNzwVDH826auSJnKHKPQS4.jsonl
⏳ Waiting for batch batch_680eb9c4b0108190ad7691e7aeaaecc8 to complete...
Batch batch_680eb9c4b0108190ad7691e7aeaaecc8 - Current status: failed
❌ Batch batch_680eb9c4b0108190ad7691e7aeaaecc8 failed or was cancelled.
⏳ Waiting for batch batch_680eb9c8c70481909ff883c0a37025fb to complete...
Batch batch_680eb9c8c70481909ff883c0a37025fb - Current status: faile

In [ ]:
# Example: peek at first few lines
file_path = "/content/drive/My Drive/batch_outputs/batch_output_file-FH1eoSrABx5psioAxKPegR.jsonl"

with open(file_path, 'r') as f:
    for i in range(5):  # check first 5 lines
        print(f.readline())


{"id": "batch_req_680ebaee89908190bbbcf69300d9ec64", "custom_id": "chunk_0", "response": {"status_code": 200, "request_id": "920953a77b9a43123f94eedb9db6c8fb", "body": {"id": "chatcmpl-BR5WT0omdJtBD9UmBbsdFAXpFXGjT", "object": "chat.completion", "created": 1745795785, "model": "gpt-4o-2024-08-06", "choices": [{"index": 0, "message": {"role": "assistant", "content": "```json\n[\n    {\n        \"query\": \"What is the version number and date of the NCCN Guidelines for Prostate Cancer mentioned in the document?\",\n        \"answer_excerpt\": \"Version 2.2025, 04/16/25\"\n    }\n]\n```", "refusal": null, "annotations": []}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 224, "completion_tokens": 57, "total_tokens": 281, "prompt_tokens_details": {"cached_tokens": 0, "audio_tokens": 0}, "completion_tokens_details": {"reasoning_tokens": 0, "audio_tokens": 0, "accepted_prediction_tokens": 0, "rejected_prediction_tokens": 0}}, "service_tier": "default", "system_finger

In [ ]:
import os
import json
import pandas as pd
import ast  # safer than eval()

# Directory where batch outputs are saved
batch_outputs_dir = "/content/drive/My Drive/batch_outputs/"

# List all downloaded jsonl files
batch_files = [os.path.join(batch_outputs_dir, f) for f in os.listdir(batch_outputs_dir) if f.endswith(".jsonl")]

gold_query_passage_pairs = []

for file_path in batch_files:
    print(f"📥 Parsing {file_path}...")
    with open(file_path, 'r') as f:
        for line in f:
            record = json.loads(line)
            if 'response' in record and 'body' in record['response']:
                choices = record['response']['body'].get('choices', [])
                if choices:
                    content = choices[0]['message'].get('content', '')
                    if content.strip():
                        try:
                            # Remove markdown formatting if necessary
                            if content.startswith("```json"):
                                content = content.replace("```json", "").replace("```", "").strip()
                            parsed = ast.literal_eval(content)
                            gold_query_passage_pairs.extend(parsed)
                        except Exception as e:
                            print(f"⚠️ Skipping bad line due to error: {e}")
                else:
                    print(f"⚠️ Empty choices in {file_path}")

print(f"✅ Total parsed query-answer pairs: {len(gold_query_passage_pairs)}")

# Save to one merged CSV
final_output_path = "/content/drive/My Drive/batch_outputs/final_ground_truth_pairs.csv"
os.makedirs("/content/drive/My Drive/batch_outputs/", exist_ok=True)
#pd.DataFrame(gold_query_passage_pairs).to_csv(final_output_path, index=False)
# Build DataFrame
# Build DataFrame
gold_df = pd.DataFrame(gold_query_passage_pairs)

# 🆕 Rename columns to match what SyntheticEvaluation expects
gold_df = gold_df.rename(columns={
    'query': 'question',
    'answer_excerpt': 'answers'
})

# 🆕 Add references as JSON list strings
import json

gold_df['references'] = gold_df['answers'].apply(lambda x: json.dumps([{"content": x}]))

# 🆕 Add dummy corpus_id
gold_df['corpus_id'] = "default_corpus"

# Save merged file
gold_df.to_csv(final_output_path, index=False)

print(f"✅ Final merged CSV saved at: {final_output_path}")


📥 Parsing /content/drive/My Drive/batch_outputs/batch_output_file-FH1eoSrABx5psioAxKPegR.jsonl...
📥 Parsing /content/drive/My Drive/batch_outputs/batch_output_file-YNzwVDH826auSJnKHKPQS4.jsonl...
✅ Total parsed query-answer pairs: 80
✅ Final merged CSV saved at: /content/drive/My Drive/batch_outputs/final_ground_truth_pairs.csv


In [ ]:
# --- Filter and Clean Using SyntheticEvaluation
corpora_paths = ['/content/drive/My Drive/ProstateCancer_2025.pdf']

evaluation = SyntheticEvaluation(corpora_paths, final_output_path, openai_api_key=OPENAI_API_KEY)

# --- Filter and Clean Using SyntheticEvaluation
evaluation.filter_poor_excerpts(threshold=0.36)
evaluation.filter_duplicates(threshold=0.6)

# Now get filtered data
filtered = evaluation.questions_df  # <--- THIS is the final data


# Select columns
cleaned_pairs = filtered[['question', 'answers']]

# Save
final_cleaned_output_path = "/content/drive/My Drive/batch_outputs/cleaned_ground_truth_pairs.csv"
cleaned_pairs.to_csv(final_cleaned_output_path, index=False)

print(f"✅ Saved final cleaned ground truth to {final_cleaned_output_path}")



Corpus: default_corpus - Removed 0 .
Corpus: default_corpus - Removed 0 .
✅ Saved final cleaned ground truth to /content/drive/My Drive/batch_outputs/cleaned_ground_truth_pairs.csv


In [ ]:
print(dir(evaluation))


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_chunker_to_collection', '_convert_question_references_to_json', '_corpus_filter_duplicates', '_corpus_filter_poor_highlights', '_extract_question_and_approx_references', '_extract_question_and_references', '_full_precision_score', '_generate_corpus_questions', '_get_chunks_and_metadata', '_get_sim', '_get_synth_questions_df', '_load_questions_df', '_save_questions_df', '_scores_from_dataset_and_retrievals', '_tag_text', 'chroma_client', 'client', 'corpora_id_paths', 'corpora_paths', 'corpus_list', 'filter_duplicates', 'filter_poor_excerpts', 'generate_queries_and_excerpts', 'is_general', 'question_maker_approx_system_prompt', 'qu

In [ ]:
# --- Preview Some Final Results
for example in cleaned_pairs.head(29).to_dict(orient='records'):
    print(example)

{'question': 'Who is the Chair of the NCCN Guidelines Panel for Prostate Cancer?', 'answers': 'Daniel E. Spratt, MD/Chair § Case Comprehensive Cancer Center/University Hospitals Seidman Cancer Center and Cleveland Clinic Taussig Cancer Institute'}
{'question': 'What is the date of the NCCN Guidelines Version 2.2025 for Prostate Cancer?', 'answers': 'Version 2.2025, 04/16/25 © 2025 National Comprehensive Cancer Network® (NCCN®)'}
{'question': 'What is a new treatment option added in the NCCN Guidelines Version 2.2025 for prostate cancer patients with PSMA-positive metastases?', 'answers': 'PROS-16 • Progression on prior hormone therapy/no prior docetaxel, useful in certain circumstances, regimen added: Lutetium Lu 177 vipivotide tetraxetan (Lu-177–PSMA-617) for PSMA-positive metastases.'}
{'question': 'What is the recommended treatment for patients with prostate cancer and life expectancy ≤ 5 years whose cancer progressed on observation of localized disease?', 'answers': 'Treatment of p

In [ ]:
cleaned_pairs.shape

(28, 2)

In [1]:
# refined method using chunker and eva library
!apt-get install -y poppler-utils
!pip install "pydantic>=2.0,<3.0"

!pip install pymupdf
!pip install langchain langchain-community faiss-cpu transformers sentence-transformers
# !pip install -U FlagEmbedding
!pip install --upgrade openai
import faiss
import torch
#from FlagEmbedding import BGEM3FlagModel
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import CrossEncoder
import numpy as np
import pandas as pd
import os



from google.colab import userdata
OPENAI_API_KEY = userdata.get('openai_API_Key')



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.7).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [2]:
from google.colab import drive

drive.mount('/content/drive/')
# Load your own PDF
pdf_path = '/content/drive/My Drive/ProstateCancer_2025.pdf'
loader = PyMuPDFLoader(pdf_path)
documents = loader.load()

print(f"Loaded {len(documents)} pages from PDF")

# Chunk your PDF
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
all_chunks = []

for doc in documents:
    page_number = doc.metadata.get('page_number', None)
    splits = splitter.split_text(doc.page_content)
    for split in splits:
        all_chunks.append({
            'text': split,
            'page_number': page_number
        })

print(f"Total {len(all_chunks)} smart chunks created.")

# 🟡 VERY IMPORTANT: Save your smart chunks into simple .txt files because SyntheticEvaluation expects file paths

# Create directory if not exist
directory = "/content/drive/My Drive/corpora"
os.makedirs(directory, exist_ok=True)

# Save all chunks into one text file (or multiple if you want)
my_text_corpus_path = os.path.join(directory, "prostate_cancer_chunks.txt")

with open(my_text_corpus_path, "w", encoding="utf-8") as f:
    for chunk in all_chunks:
        f.write(chunk['text'] + "\n\n")  # Separate chunks a bit

print(f"Saved all chunks into {my_text_corpus_path}")

# # ✅ Now use SyntheticEvaluation exactly like before, but on your text file
# corpora_paths = [my_text_corpus_path]
# queries_csv_path = '/content/drive/My Drive/corpora/generated_queries_and_excerpts.csv'





Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Loaded 223 pages from PDF
Total 1895 smart chunks created.
Saved all chunks into /content/drive/My Drive/corpora/prostate_cancer_chunks.txt


restart the session and run the cell below. Because langchang and chunking uses different version of Pydantic

In [ ]:
# --------------------------------------------
# ✅ STEP 1: Install Dependencies (modern OpenAI version compatible)
# --------------------------------------------
!pip install --upgrade openai
!pip install chromadb==0.4.11
!pip install git+https://github.com/brandonstarxel/chunking_evaluation.git

# --------------------------------------------
# ✅ STEP 2: Set OpenAI Key
# --------------------------------------------
from google.colab import userdata
import os

OPENAI_API_KEY = userdata.get("openai_API_Key")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
!sed -i '/from chromadb.api.types import (/,/)/d' /usr/local/lib/python3.11/dist-packages/chunking_evaluation/chunking/kamradt_modified_chunker.py
# --------------------------------------------
# ✅ STEP 3: Define Modern OpenAI Embedding Wrapper
# --------------------------------------------
from openai import OpenAI
from chromadb.api.types import EmbeddingFunction, Documents, Embeddings

class ModernOpenAIEmbeddingFunction(EmbeddingFunction):
    def __init__(self, api_key: str, model_name: str = "text-embedding-3-large"):
        self.client = OpenAI(api_key=api_key)
        self.model_name = model_name

    def __call__(self, texts: Documents) -> Embeddings:
        if isinstance(texts, str):
            texts = [texts]
        response = self.client.embeddings.create(
            input=texts,
            model=self.model_name,
        )
        return [r.embedding for r in response.data]

embedding_fn = ModernOpenAIEmbeddingFunction(api_key=OPENAI_API_KEY)
from google.colab import drive

drive.mount('/content/drive/')
# --------------------------------------------
# ✅ STEP 4: Initialize SyntheticEvaluation
# --------------------------------------------
from chunking_evaluation import SyntheticEvaluation, BaseChunker

corpus_path = "/content/drive/My Drive/corpora/prostate_cancer_chunks.txt"
queries_csv_path = "/content/drive/My Drive/corpora/generated_queries_and_excerpts.csv"

evaluation = SyntheticEvaluation(
    corpora_paths=[corpus_path],
    queries_csv_path=queries_csv_path,
    openai_api_key=OPENAI_API_KEY
)

# --------------------------------------------
# ✅ STEP 5: Generate & Filter Questions
# --------------------------------------------
evaluation.generate_queries_and_excerpts(
    approximate_excerpts=True,  # optional
    num_rounds=1,               # repeat sampling 2 times
    queries_per_corpus=120       # generate 30 per round
)
evaluation.filter_poor_excerpts(threshold=0.36)
evaluation.filter_duplicates(threshold=0.6)
# Now get filtered data
filtered = evaluation.questions_df  # <--- THIS is the final data


# Select columns
cleaned_pairs = filtered[['question', 'answers']]

# Save
final_cleaned_output_path = "/content/drive/My Drive/batch_outputs/cleaned_ground_truth_pairs_100.csv"
cleaned_pairs.to_csv(final_cleaned_output_path, index=False)

print(f"✅ Saved final cleaned ground truth to {final_cleaned_output_path}")


  Cloning https://github.com/brandonstarxel/chunking_evaluation.git to /tmp/pip-req-build-s17mj6vo
  Running command git clone --filter=blob:none --quiet https://github.com/brandonstarxel/chunking_evaluation.git /tmp/pip-req-build-s17mj6vo
  Resolved https://github.com/brandonstarxel/chunking_evaluation.git to commit d451fc4cf56e417b755994b4ca5212fd5057c0d2
  Preparing metadata (setup.py) ... done
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Trying Query 0
Trying Query 1
Trying Query 2
Trying Query 3
Trying Query 4
Trying Query 5
Trying Query 6
Trying Query 7
Trying Query 8
Trying Query 9
Trying Query 10
Trying Query 11
Trying Query 12
Trying Query 13
Trying Query 14
Trying Query 15
Trying Query 16
Trying Query 17
Trying Query 18
Trying Query 19
Trying Query 20
Trying Query 21
Trying Query 22
Trying Query 23
Trying Query 24
Trying Query 25
Trying Query 26
Trying Query 27
Trying Query 28
Trying Query 2

In [9]:
# Select columns
cleaned_pairs = filtered[['question', 'references']]

# Save
final_cleaned_output_path = "/content/drive/My Drive/batch_outputs/cleaned_ground_truth_pairs_100.csv"
cleaned_pairs.to_csv(final_cleaned_output_path, index=False)

print(f"✅ Saved final cleaned ground truth to {final_cleaned_output_path}")


✅ Saved final cleaned ground truth to /content/drive/My Drive/batch_outputs/cleaned_ground_truth_pairs_100.csv


In [7]:
filtered.head(20)

,question,references,corpus_id
0,What benefits did the phase 3 clinical trial f...,[{'content': 'prostate cancer/NEPC. ◊Sipuleuce...,/content/drive/My Drive/corpora/prostate_cance...
1,What are the primary sources for information o...,[{'content': 'Available at: http://www.ncbi.nl...,/content/drive/My Drive/corpora/prostate_cance...
2,What are the considerations for transitioning ...,[{'content': 'clinical trials. PSADT is not co...,/content/drive/My Drive/corpora/prostate_cance...
3,What are the regimen options for docetaxel in ...,[{'content': 'FDA-approved for metastatic CRPC...,/content/drive/My Drive/corpora/prostate_cance...
4,What are the cumulative 7-year rates of bioche...,[{'content': 'ti-institutional phase 2 trials ...,/content/drive/My Drive/corpora/prostate_cance...
5,Which study discussed the comparison of late g...,[{'content': '2008;70:1124-1129. Available at:...,/content/drive/My Drive/corpora/prostate_cance...
6,What was the overall impact of abiraterone on ...,[{'content': 'overall discontinuation rate du...,/content/drive/My Drive/corpora/prostate_cance...
7,What are the key findings on the effect of ora...,"[{'content': 'Saad F, Cookson MS, et al. Oral ...",/content/drive/My Drive/corpora/prostate_cance...
8,What are the restrictions on reproducing NCCN ...,"[{'content': '(NCCN©), All rights reserved. NC...",/content/drive/My Drive/corpora/prostate_cance...
9,What are the publication years for the prostat...,[{'content': 'Long-term outcome for very high-...,/content/drive/My Drive/corpora/prostate_cance...


In [ ]:

# --------------------------------------------
# ✅ STEP 6: Define Chunker
# --------------------------------------------
class CustomChunker(BaseChunker):
    def split_text(self, text):
        return [text[i:i+1200] for i in range(0, len(text), 1200)]

chunker = CustomChunker()

# --------------------------------------------
# ✅ STEP 7: Run Evaluation using modern embedding
# --------------------------------------------
results = evaluation.run(chunker, embedding_function=embedding_fn)
print("✅ Evaluation Results:", results)